In [4]:
!git add bmw_quantum_challenge

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   bmw_quantum_challenge.ipynb
	new file:   metrics/resnet/avg_acc_test.npy
	new file:   metrics/resnet/avg_acc_train.npy
	new file:   metrics/resnet/avg_loss_test.npy
	new file:   metrics/resnet/avg_loss_train.npy
	new file:   metrics/resnet_q_angular_depth1/avg_acc_test.npy
	new file:   metrics/resnet_q_angular_depth1/avg_acc_train.npy
	new file:   metrics/resnet_q_angular_depth1/avg_loss_test.npy
	new file:   metrics/resnet_q_angular_depth1/avg_loss_train.npy
	new file:   metrics/resnet_q_angular_depth5/avg_acc_test.npy
	new file:   metrics/resnet_q_angular_depth5/avg_acc_train.npy
	new file:   metrics/resnet_q_angular_depth5/avg_loss_test.npy
	new file:   metrics/resnet_q_angular_depth5/avg_loss_train.npy

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discar

In [ ]:
!pip install torchvision
!pip install sagemaker

### Import stuff

In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np

import torch
from torch import optim

from torchvision import datasets
from torchvision import transforms
import torchvision.models as models

from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

import torch.nn as nn
import pathlib

import random

### load data from S3 to folder of sagemaker instance

In [ ]:
import boto3

# When running on SageMaker, need execution role
from sagemaker import get_execution_role
role = get_execution_role()

# Declare bucket name, remote file, and destination
s3_bucket = "amazon-braket-a79a20b5e0b4"
orig_file = 'defect_data.zip'
dest_file = './defect_data.zip'

# Connect to S3 bucket and download file
s3 = boto3.resource('s3')
s3.Bucket(s3_bucket).download_file(orig_file, dest_file)

### unzip defect_data.zip 

In [ ]:
from zipfile import ZipFile

# Create a ZipFile Object and load sample.zip in it
with ZipFile('defect_data.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall('./defect_data')

### Data preparation 

In [ ]:
def get_train_test_data(img_path, img_set_size = 40e3, test_set_ratio = 0.2):
    """
    Create train and test data with PyTorchs ImageFolder class.  
    """

    resolution = (227,227)
    
    img_dataset = datasets.ImageFolder(
        root=img_path,
        transform=transforms.Compose([
            transforms.Resize(resolution), # do not have to resize because all imgs are on 227x227
            transforms.ToTensor(),
            # maps values from [0, 1] to [-1,1]
            # 0.5 mean and 0.5 std deviation for each channel is a standard choice
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
        ]))
    
    # restrict size of data set
    # take n random indices out of full img_dataset size
    random_indices = random.sample(range(len(img_dataset)), img_set_size)

    img_dataset_subset = torch.utils.data.Subset(img_dataset, random_indices)
    
    # split data set into train and test set
    test_set_size = int(test_set_ratio * len(img_dataset_subset))
    train_set_size = len(img_dataset_subset) - test_set_size   
    train_set, test_set = random_split(img_dataset_subset, [train_set_size, test_set_size], generator=torch.Generator().manual_seed(42))
    
    # with attribute class_to_idx we get a mapping from classes to labels. We invert the dict subsequently.
    label_to_class_dict = {value: key for key, value in img_dataset.class_to_idx.items()}

    return train_set, test_set, label_to_class_dict

### load data set

In [ ]:
img_path = './defect_data'
train_set, test_set, label_to_class_dict = get_train_test_data(img_path, img_set_size = 100)


BATCH_SIZE = 32 

train_loader = DataLoader(
    train_set,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_loader = DataLoader(
    test_set,
    batch_size=BATCH_SIZE,
    shuffle=False
)


In [ ]:
len(train_set)

In [ ]:
len(test_set)

### train functions

In [ ]:
def get_plain_accuracy(y_pred, y_true):
    return (y_pred.argmax(dim=1) == y_true).float().mean()


def eval_test_set(model, best_loss, model_name, run_idx):
    print("### Eval test set ###")

    model.eval()
    loss_fn = torch.nn.CrossEntropyLoss(reduction = 'mean')
    
    y_test_pred = torch.tensor([]).float()
    y_test_true = torch.tensor([]).long()  
    
    # forward pass is faster with torch.no_grad()
    with torch.no_grad(): 
            
        for x_test_batch, y_test_true_batch in test_loader:
            
            y_test_pred_batch = model(x_test_batch)
            
            y_test_pred = torch.cat([y_test_pred, y_test_pred_batch])
            y_test_true = torch.cat([y_test_true, y_test_true_batch])            
                    
    loss_test = loss_fn(y_test_pred, y_test_true)
    acc_test = get_plain_accuracy(y_test_pred, y_test_true)

    print(f"Loss test: {loss_test:.3f}")
    print(f"Accuracy test: {acc_test:.3f}")  

    # save best_model
    if loss_test < best_loss:
        state = {
            'state_dict': model.state_dict()
        }
        torch.save(state, f'./models/{model_name}_best_model_run_{run_idx}.h5')
        best_loss = loss_test
        
    print("#####################\n")    
    return best_loss, loss_test, acc_test  


def train(model, num_epochs, run_idx, model_name="resnet"):
    """Train function"""
    

    # define loss and optimizer
    loss_fn = torch.nn.CrossEntropyLoss(reduction = 'mean')
    optimizer = optim.Adam(model.parameters())

    acc_train_list = []
    loss_train_list = []    
    
    acc_test_list = []    
    loss_test_list = []   
    
    best_loss = 10e3
    
    for epoch in range(num_epochs):

        print(f'### Epoch {epoch} ###\n')

        for idx, (x_train, y_true) in enumerate(train_loader):

            model.train()
            
            print(f'Batch iter/total {idx}/{len(train_loader)-1}')
            # zero the gradients
            optimizer.zero_grad()

            # do one forward pass
            y_pred = model(x_train)             

            # calculate the loss
            loss = loss_fn(y_pred, y_true)
            acc_train = get_plain_accuracy(y_pred, y_true)

            print(f"Loss train: {loss:.3f}")
            print(f"Accuracy train: {acc_train:.3f}\n")

            # get gradients with respect to that loss
            loss.backward()

            # actual optimizing step
            optimizer.step()

            acc_train_list.append(acc_train.item())
            loss_train_list.append(loss.item())
        
            # eval test set every 5 iters
            if ((idx + 1) % 5) == 0:
                best_loss, loss_test, acc_test = eval_test_set(model, best_loss, model_name,run_idx)
                
                acc_test_list.append(acc_test.item())
                loss_test_list.append(loss_test.item())
                
    return acc_train_list, acc_test_list, loss_train_list, loss_test_list


def plot_stuff(data_, title, plot_each_metric=False):
    
    data = data_.mean(axis=0)
    std = data_.std(axis=0)
    
    plt.figure(figsize=(9,5))
    plt.title(title)
    plt.plot(data)
    
    if plot_each_metric:
        plt.plot(data_.T)
    
    plt.fill_between(range(len(data)), data - std, data + std, color='tab:blue', alpha=0.15)
                 
    plt.grid()
    plt.show()

### load pretrained resnet 18 and freeze all conv layers

In [ ]:

def run_experiment(num_runs, num_epochs, nn_type, q_depth=None):
    # start training process and measure time duration
    start_time = time.time()

    avg_acc_train = np.array([])
    avg_acc_test = np.array([])

    avg_loss_train = np.array([])
    avg_loss_test = np.array([])

    for run_idx in range(num_runs):

        # load on ImageNet pretrained resnet18
        model = models.resnet18(pretrained=True)

        # freeze all layers
        for name, param in model.named_parameters():
            param.requires_grad = False

        # attach new last classic or quantum layer
        if nn_type == 'classic':
            model.fc = nn.Linear(512, 2)  
            model_name="resnet"
            
        elif nn_type == 'angular':
            model.fc = QuantumNetAngularEmbedding(q_depth = q_depth)
            model_name=f"resnet_q_angular_depth{q_depth}"
            
        elif nn_type == 'amplitude':
            model.fc = QuantumNetAmplitudeEmbedding(n_qubits=9, q_depth=q_depth)
            model_name=f"resnet_q_amplitude_depth{q_depth}"    
            
        if  run_idx == 0:  
            print(f'########## {model_name} ##########')
        print(f'\n########## run {run_idx} ##########\n')

        acc_train, acc_test, loss_train, loss_test = train(model=model, num_epochs = num_epochs, run_idx= run_idx,model_name=model_name)

        avg_acc_train = np.vstack([avg_acc_train, np.array(acc_train)]) if avg_acc_train.size else np.array(acc_train)  
        avg_acc_test = np.vstack([avg_acc_test, np.array(acc_test)]) if avg_acc_test.size else np.array(acc_test)  

        avg_loss_train = np.vstack([avg_loss_train, np.array(loss_train)]) if avg_loss_train.size else np.array(loss_train)  
        avg_loss_test = np.vstack([avg_loss_test, np.array(loss_test)]) if avg_loss_test.size else np.array(loss_test)        

    total_time = time.time() - start_time

    print(f'total time taken {total_time:.2f}\n')
    
    
    # create folder if not exists
    pathlib.Path(f"./metrics/{model_name}").mkdir(parents=True, exist_ok=True)
    
    # store metrics
    np.save(f'./metrics/{model_name}/avg_acc_test.npy', avg_acc_test)
    np.save(f'./metrics/{model_name}/avg_acc_train.npy', avg_acc_train)

    np.save(f'./metrics/{model_name}/avg_loss_train.npy', avg_loss_train)
    np.save(f'./metrics/{model_name}/avg_loss_test.npy', avg_loss_test)
    
    return avg_acc_train.T, avg_acc_test.T, avg_loss_train.T, avg_loss_test.T
    
    

## Train CNN classically

In [ ]:
num_runs = 3
num_epochs = 4

In [ ]:

avg_acc_train, avg_acc_test, avg_loss_train, avg_loss_test = run_experiment(num_runs, num_epochs, nn_type='classic')

In [ ]:

avg_acc_test = np.load('./metrics/resnet/avg_acc_test.npy')
avg_acc_train = np.load('./metrics/resnet/avg_acc_train.npy')

avg_loss_train = np.load('./metrics/resnet/avg_loss_train.npy')
avg_loss_test = np.load('./metrics/resnet/avg_loss_test.npy')

print(f'max average test accuracy {max(avg_acc_test.mean(axis=0)):.3f}')

plot_stuff(avg_acc_test, title = 'Acc test')
plot_stuff(avg_acc_train, title = 'Acc train')

plot_stuff(avg_loss_train, title = 'Loss train')
plot_stuff(avg_loss_test, title = 'Loss test')


### Quantum Net 

In [ ]:
# Pennylane
import pennylane as qml
from pennylane import numpy as np

In [ ]:
# hyperparameters of quantum net

n_qubits = 4        

init_param_spread = 0.01 # Initial spread of random quantum weights

dev = qml.device("braket.local.qubit", wires=n_qubits)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:


def entangling_layer(n_qubits = n_qubits, pattern = 'chain'):
    """pattern in ('chain', 'ring', 'all_to_all')"""
    
    if pattern not in ('chain', 'ring', 'all_to_all', 'random'):
        pattern = 'chain'
    if pattern is not 'random':
        qml.broadcast(qml.CNOT, wires = list(range(n_qubits)), pattern = pattern)
    else:
        q_idx_list = list(range(n_qubits))
        np.random.shuffle(q_idx_list)
        for i in range(n_qubits - 1):
            qml.CNOT(wires = [q_idx_list[i], q_idx_list[i + 1]])
        qml.CNOT(wires = [q_idx_list[-1], q_idx_list[0]])

def H_layer(nqubits):
    """Layer of single-qubit Hadamard gates.
    """
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)


def RY_layer(w):
    """Layer of parametrized qubit rotations around the y axis.
    """
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)

def amplitude_embedding(q_input_features, n_qubits = n_qubits):
    """Transfers a given input feature into a quantum statevector. 2**9 = 512"""

    
    assert len(q_input_features) == 2**n_qubits, "Input Features must be of dimension 2**n_qubits"
    
    wires_list = list(range(n_qubits))
    
    # TODO: replace equality comparison to one with tolerance
    normalize = not (np.linalg.norm(q_input_features) == 1)
            
    qml.templates.AmplitudeEmbedding(features = q_input_features, 
                                     wires = wires_list, 
                                     normalize = normalize) 
        
@qml.qnode(dev, interface="torch")
def quantum_net(q_input_features, q_weights_flat):
    """
    The variational quantum circuit.
    """

    # Reshape weights
    q_weights = q_weights_flat.reshape(q_depth, n_qubits)

    # Start from state |+> , unbiased w.r.t. |0> and |1>
    H_layer(n_qubits)

    # Embed features in the quantum node
    RY_layer(q_input_features)

    # Sequence of trainable variational layers
    for k in range(q_depth):
        entangling_layer(n_qubits = n_qubits, pattern = 'ring')
        RY_layer(q_weights[k])

    # Expectation values in the Z basis
    exp_vals = [qml.expval(qml.PauliZ(position)) for position in range(n_qubits)]
    return tuple(exp_vals)

@qml.qnode(dev, interface="torch")
def quantum_net_amplitude(q_input_features, q_weights_flat):
    """
    The variational quantum circuit with amplitude embedding
    """

    # Reshape weights
    q_weights = q_weights_flat.reshape(q_depth, n_qubits)

    amplitude_embedding(q_input_features, n_qubits)

    # Sequence of trainable variational layers
    for k in range(q_depth):
        entangling_layer(n_qubits = n_qubits, pattern = 'ring')
        RY_layer(q_weights[k])

    # Expectation values in the Z basis
    exp_vals = [qml.expval(qml.PauliZ(position)) for position in range(n_qubits)]
    
    # Why do we return tuple here?
    return tuple(exp_vals)



#### draw circuit

In [ ]:
q_depth = 1
n_qubits = 4
q_input_features = (torch.rand(n_qubits) * 2 - 1) * np.pi / 2
q_weights_flat = torch.randn(q_depth * n_qubits)

# evaluation of the circuit is necessary to draw the circuit 
result = quantum_net(q_input_features, q_weights_flat)

print(quantum_net.draw())

In [ ]:
# amplitude embedding 
n_qubits = 9
q_input_features = torch.randn(2 ** n_qubits)

q_weights_flat = torch.randn(q_depth * n_qubits)

# evaluation of the circuit is necessary to draw the circuit 
dev = qml.device("braket.local.qubit", wires=n_qubits)

print(q_input_features.shape)

result = quantum_net_amplitude(q_input_features, q_weights_flat)

# print(quantum_net_amplitude.draw())

# result

In [ ]:
class QuantumNetAngularEmbedding(nn.Module):
    """
    Torch module implementing the *dressed* quantum net.
    """

    def __init__(self, q_depth):
        """
        Definition of the *dressed* layout.
        """

        super().__init__()
        self.pre_net = nn.Linear(512, n_qubits)
        self.q_params = nn.Parameter(init_param_spread * torch.randn(q_depth * n_qubits))
        self.post_net = nn.Linear(n_qubits, 2)

    def forward(self, input_features):
        """
        Defining how tensors are supposed to move through the *dressed* quantum
        net.
        """

        # obtain the input features for the quantum circuit
        # by reducing the feature dimension from 512 to 4
        pre_out = self.pre_net(input_features)
        q_in = torch.tanh(pre_out) * np.pi / 2.0

        # Apply the quantum circuit to each element of the batch and append to q_out
        q_out = torch.Tensor(0, n_qubits)
        q_out = q_out.to(device)
        for elem in q_in:
            q_out_elem = quantum_net(elem, self.q_params).float().unsqueeze(0)
            q_out = torch.cat((q_out, q_out_elem))

        # return the two-dimensional prediction from the postprocessing layer
        return self.post_net(q_out)
    
class QuantumNetAmplitudeEmbedding(nn.Module):
    """
    Torch module implementing the *dressed* quantum net.
    """

    def __init__(self, n_qubits, q_depth):
        """
        Definition of the *dressed* layout.
        """

        super().__init__()
        
        self.q_params = nn.Parameter(init_param_spread * torch.randn(q_depth * n_qubits))

    def forward(self, input_features):
        """
        Defining how tensors are supposed to move through the *dressed* quantum
        net.
        """

        # Apply the quantum circuit to each element of the batch and append to q_out
        q_out = torch.Tensor(0, n_qubits )
        q_out = q_out.to(device)
        
        # iterate over each element of the batch
        for elem in input_features:
            q_out_elem = quantum_net_amplitude(elem, self.q_params).float().unsqueeze(0)
            q_out = torch.cat((q_out, q_out_elem))

        # restrict to first two dimensions and ignore the rest
        return q_out[...,:2]

### Train Quantum CNN with angular embedding for different depth sizes

In [ ]:

for q_depth in [1]:

    q_avg_acc_train, q_avg_acc_test, q_avg_loss_train, q_avg_loss_test = run_experiment(num_runs, num_epochs, nn_type = 'angular', q_depth = q_depth)

In [ ]:
for q_depth in [1,5]:
    
    model_name=f"resnet_q_angular_depth{q_depth}"
    
    # load quantum metrics
    q_avg_acc_test = np.load(f'./metrics/{model_name}/avg_acc_test.npy')
    q_avg_acc_train = np.load(f'./metrics/{model_name}/avg_acc_train.npy')

    q_avg_loss_train = np.load(f'./metrics/{model_name}/avg_loss_train.npy')
    q_avg_loss_test = np.load(f'./metrics/{model_name}/avg_loss_test.npy')
    
    print(f'#### DEPTH {q_depth} ####')
#     print(q_avg_acc_test)
    
    print(f'max average test accuracy {max(q_avg_acc_test.mean(axis=1)):.3f}')

    plot_stuff(q_avg_acc_test, title = 'Acc test',plot_each_metric=False)
    plot_stuff(q_avg_loss_test, title = 'Loss test',plot_each_metric=False)
    
    plot_stuff(q_avg_acc_train, title = 'Acc train',plot_each_metric=False)
    plot_stuff(q_avg_loss_train, title = 'Loss train',plot_each_metric=False)




### Plot everything combined 

In [ ]:
def plot_stuff_combined(data_list, labels_list, title):
    
    plt.figure(figsize=(12,7))
    for idx, data_ in enumerate(data_list):

        data = data_.mean(axis=0)
        std = data_.std(axis=0)

        plt.title(title)

        plt.plot(data, label=labels_list[idx])

        plt.fill_between(range(len(data)), data - std, data + std, alpha=0.15)
        
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
# load classic metrics 
avg_acc_test = np.load('./metrics/resnet/avg_acc_test.npy')
avg_acc_train = np.load('./metrics/resnet/avg_acc_train.npy')

avg_loss_train = np.load('./metrics/resnet/avg_loss_train.npy')
avg_loss_test = np.load('./metrics/resnet/avg_loss_test.npy')

acc_test_list = []
acc_train_list = []

loss_test_list = []
loss_train_list = []

labels_list = []

acc_test_list.append(avg_acc_test)
acc_train_list.append(avg_acc_train)

loss_test_list.append(avg_loss_test)
loss_train_list.append(avg_loss_train)

labels_list.append("Classic")


for q_depth in [1,5]:
    
    model_name=f"resnet_q_angular_depth{q_depth}"
    
    print(f'########## {model_name} ##########')

    # load quantum metrics
    avg_acc_test = np.load(f'./metrics/{model_name}/avg_acc_test.npy')
    avg_acc_train = np.load(f'./metrics/{model_name}/avg_acc_train.npy')

    avg_loss_train = np.load(f'./metrics/{model_name}/avg_loss_train.npy')
    avg_loss_test = np.load(f'./metrics/{model_name}/avg_loss_test.npy')
    
    acc_test_list.append(avg_acc_test)
    acc_train_list.append(avg_acc_train)

    loss_test_list.append(avg_loss_test)
    loss_train_list.append(avg_loss_train)
    
    labels_list.append(f"Q_Resnet_depth_{q_depth}")

# plot in one plot
plot_stuff_combined(acc_test_list, labels_list, title = 'Acc test')
plot_stuff_combined(loss_test_list, labels_list, title = 'Loss test')

plot_stuff_combined(acc_train_list, labels_list, title = 'Acc train')
plot_stuff_combined(loss_train_list, labels_list, title = 'Loss train')

### Train QNN with amplitude embedding

In [ ]:
num_runs = 2
num_epochs = 2
for q_depth in [1]:

    q_avg_acc_train, q_avg_acc_test, q_avg_loss_train, q_avg_loss_test = run_experiment(num_runs, num_epochs, nn_type = 'amplitude', q_depth = q_depth)

In [ ]:
!git status
